In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# How much data do we have?
import codecs
import glob

texts = []

txt_files = glob.glob('/home/jvdzwaan/data/ocr/*.gs.txt')
#txt_files = glob.glob('/home/jvdzwaan/data/ocr/BAO*.gs.txt')  # Articles from AD only
print 'Number of newspaper articles:', len(txt_files)
for txt_file in txt_files:
    with codecs.open(txt_file, encoding='utf-8') as f:
        text = f.read()
    texts.append(text)

seq_length = 100
#raw_text = ('\n'*seq_length).join(texts)
raw_text = u' '.join(texts)
with codecs.open('/home/jvdzwaan/data/ocr/all_gs.txt', 'w', encoding='utf-8') as f:
    #f.write('\n'*seq_length)
    f.write(raw_text)
    #f.write('\n'*seq_length)

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))    

n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

In [ ]:
print chars

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# define the LSTM model (3 layers of 512 nodes each)
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="/home/jvdzwaan/data/tmp/dncvu-lstm-256/{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
print X.shape
print y.shape

In [ ]:
model.fit(X, y, epochs=20, batch_size=100, callbacks=callbacks_list)

In [ ]:
# load the network weights
filename = "/home/jvdzwaan/data/tmp/dncvu-lstm/16-1.7333.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
int_to_char[3]

In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
text = []
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    #print prediction[0]
    #print prediction.shape
    #index = numpy.argmax(prediction)
    index = numpy.random.choice(n_vocab, p=prediction[0])
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    #print(result)
    text.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."
print u''.join(text)

In [ ]:
import codecs

with codecs.open('/home/jvdzwaan/data/ocr/BNObi1-ds.ocr.txt', encoding='utf-8') as f:
    ocr_text = f.read()
with codecs.open('/home/jvdzwaan/data/ocr/BNObi1-ds.gs.txt', encoding='utf-8') as f:
    gs_text = f.read()

print ocr_text
print
print gs_text

In [ ]:
n_chars = len(ocr_text)
seq_length = 5
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = ocr_text[i:i + seq_length]
    seq_out = ocr_text[i + seq_length]
    print seq_in, gs_text[i:i + seq_length]